#### Chapter 6
# Data Loading, Storage, and File Formats

- Đọc dữ liệu và làm cho nó có thể truy cập được (thường gọi là nạp dữ liệu) là bước đầu tiên cần thiết để sử dụng hầu hết các công cụ trong cuốn sách này. 
- Thuật ngữ phân tích cú pháp (parsing) đôi khi cũng được dùng để mô tả việc nạp dữ liệu dạng văn bản và diễn giải chúng thành các bảng và các kiểu dữ liệu khác nhau. 
- Tôi sẽ tập trung vào việc nhập và xuất dữ liệu bằng pandas, mặc dù còn có rất nhiều công cụ trong các thư viện khác hỗ trợ đọc và ghi dữ liệu ở nhiều định dạng khác nhau.

- Việc nhập và xuất dữ liệu thường rơi vào một số loại chính: 
    - đọc các tệp văn bản và các định dạng lưu trữ hiệu quả hơn trên đĩa
    - nạp dữ liệu từ cơ sở dữ liệu, 
    - tương tác với các nguồn mạng như web APIs.

## 6.1 Reading and Writing Data in Text Format

- pandas có nhiều hàm để đọc dữ liệu dạng bảng thành đối tượng DataFrame. Bảng 6-1 tóm tắt một số hàm trong số đó; 
- Trong cuốn sách này, pandas.read_csv là một trong những hàm được sử dụng thường xuyên nhất. 

> Chúng ta sẽ tìm hiểu về các định dạng dữ liệu nhị phân ở Mục 6.2, “Định dạng dữ liệu nhị phân,” tại trang 193.

> Table 6-1. Text and binary data loading functions in pandas

| Function       | Description                                                                                   |
|----------------|-----------------------------------------------------------------------------------------------|
| read_csv       | Load delimited data from a file, URL, or file-like object; use comma as default delimiter     |
| read_fwf       | Read data in fixed-width column format (i.e., no delimiters)                                  |
| read_clipboard | Variation of read_csv that reads data from the clipboard; useful for converting tables from web pages |
| read_excel     | Read tabular data from an Excel XLS or XLSX file                                              |
| read_hdf       | Read HDF5 files written by pandas                                                             |
| read_html      | Read all tables found in the given HTML document                                              |
| read_json      | Read data from a JSON string, file, URL, or file-like object                                  |
| read_feather   | Read the Feather binary file format                                                           |
| read_orc       | Read the Apache ORC binary file format                                                        |
| read_parquet   | Read the Apache Parquet binary file format                                                    |
| read_pickle    | Read an object stored by pandas using the Python pickle format                                |
| read_sas       | Read a SAS dataset stored in one of the SAS system’s custom storage formats                   |
| read_spss      | Read a data file created by SPSS                                                              |
| read_sql       | Read the results of a SQL query (using SQLAlchemy)                                            |
| read_sql_table | Read a whole SQL table (using SQLAlchemy); equivalent to using a query that selects everything |
| read_stata     | Read a dataset from Stata file format                                                         |
| read_xml       | Read a table of data from an XML file                                                         |


- Tôi sẽ đưa ra phần tổng quan về cơ chế hoạt động của các hàm này, vốn được dùng để chuyển đổi dữ liệu dạng văn bản thành một DataFrame. 
- Các tham số tùy chọn cho những hàm này có thể rơi vào một vài nhóm sau:

> Indexing

- Có thể coi một hoặc nhiều cột là DataFrame trả về, và quyết định có lấy tên cột từ tệp, từ các tham số bạn cung cấp, hoặc không lấy tên cột nào cả.

> Type inference and data conversion

- Bao gồm các chuyển đổi giá trị do người dùng định nghĩa và danh sách tùy chỉnh cho các giá trị bị thiếu.

> Date and time parsing

- Hỗ trợ khả năng kết hợp, chẳng hạn kết hợp thông tin ngày và giờ nằm rải rác ở nhiều cột thành một cột duy nhất trong kết quả.

> Iterating

- Hỗ trợ việc lặp qua các khối dữ liệu nhỏ của những tệp rất lớn.

> Unclean data issues

- Bao gồm bỏ qua các hàng hoặc phần chân (footer), chú thích, hoặc những vấn đề nhỏ khác như dữ liệu số có dấu phẩy phân tách hàng nghìn.

Bởi vì dữ liệu trong thế giới thực thường rất lộn xộn, một số hàm nạp dữ liệu (đặc biệt là pandas.read_csv) theo thời gian đã tích lũy một danh sách dài các tham số tùy chọn. <br>
Việc cảm thấy choáng ngợp trước số lượng tham số khác nhau là điều bình thường (pandas.read_csv có khoảng 50 tham số). <br>
Tài liệu trực tuyến của pandas có rất nhiều ví dụ về cách hoạt động của từng tham số, vì vậy nếu bạn gặp khó khăn khi đọc một tệp cụ thể, có thể sẽ có một ví dụ tương tự đủ gần để giúp bạn tìm ra các tham số phù hợp.
<br><br>
Một số hàm trong số này thực hiện suy luận kiểu dữ liệu, bởi vì kiểu dữ liệu của các cột không nằm trong định dạng dữ liệu. 
<br>Điều đó có nghĩa là bạn không nhất thiết phải chỉ định cột nào là số thực, số nguyên, Boolean hay chuỗi. Các định dạng dữ liệu khác, như HDF5, ORC và Parquet, có thông tin về kiểu dữ liệu được nhúng sẵn trong chính định dạng.

- Việc xử lý ngày tháng và các kiểu dữ liệu tùy chỉnh khác có thể cần thêm công sức.

In [5]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))
pd.options.display.max_colwidth = 75
pd.options.display.max_columns = 20
np.set_printoptions(precision=4, suppress=True)

> Hãy bắt đầu với một tệp văn bản CSV (comma-separated values) nhỏ:

In [6]:
!cat examples/ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

> Vì đây là dữ liệu được phân tách bằng dấu phẩy, nên chúng ta có thể dùng pandas.read_csv để đọc nó vào một DataFrame:

In [7]:
df = pd.read_csv("examples/ex1.csv")

In [8]:
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


> Một tệp không phải lúc nào cũng có hàng tiêu đề. Hãy xem xét tệp sau:

In [9]:
!cat examples/ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

- Để đọc tệp này, bạn có một vài lựa chọn. Bạn có thể để pandas tự gán tên cột mặc định, hoặc bạn có thể tự chỉ định tên cột:

In [10]:
pd.read_csv("examples/ex2.csv", header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [11]:
pd.read_csv("examples/ex2.csv", names=["a", "b", "c", "d", "message"])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


> Giả sử bạn muốn cột message trở thành chỉ mục (index) của DataFrame trả về. Bạn có thể chỉ định cột ở vị trí số 4 hoặc cột có tên "message" bằng cách sử dụng tham số index_col:

In [12]:
names = ["a", "b", "c", "d", "message"]
pd.read_csv("examples/ex2.csv", names=names, index_col="message")

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


- Nếu bạn muốn tạo một chỉ mục phân cấp (hierarchical index) (sẽ được thảo luận trong Mục 8.1, “Đánh chỉ mục phân cấp,” ở trang 247) từ nhiều cột, hãy truyền vào một danh sách các số thứ tự cột hoặc tên cột:

In [13]:
!cat examples/csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [14]:
parsed = pd.read_csv("examples/csv_mindex.csv",
                     index_col=["key1", "key2"])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

- Trong một số trường hợp, một bảng có thể không có ký tự phân tách cố định, mà dùng khoảng trắng hoặc một mẫu ký tự khác để tách các trường. Hãy xem xét một tệp văn bản có dạng như sau:

In [15]:
!cat examples/ex3.txt

            A         B         C
aaa -0.264438 -1.026059 -0.619500
bbb  0.927272  0.302904 -0.032399
ccc -0.264273 -0.386314 -0.217601
ddd -0.871858 -0.348382  1.100491


- Mặc dù bạn có thể tự xử lý thủ công, nhưng trong trường hợp này các trường được tách bằng một lượng khoảng trắng không cố định. Với những tình huống như vậy, bạn có thể truyền một biểu thức chính quy (regular expression) làm ký tự phân tách cho pandas.read_csv. Biểu thức này có thể được viết là \s+, và khi đó ta sẽ có:

In [16]:
result = pd.read_csv("examples/ex3.txt", sep=r"\s+")
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


- Bởi vì số lượng tên cột ít hơn số cột dữ liệu một cột, nên trong trường hợp đặc biệt này, pandas.read_csv sẽ suy luận rằng cột đầu tiên nên được dùng làm chỉ mục (index) của DataFrame.

- Các hàm phân tích tệp có rất nhiều tham số bổ sung để giúp bạn xử lý sự đa dạng của các định dạng tệp ngoại lệ (xem danh sách một phần trong Bảng 6-2). Ví dụ, bạn có thể bỏ qua hàng đầu tiên, hàng thứ ba và hàng thứ tư của một tệp bằng tham số skiprows:

In [17]:
!cat examples/ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [18]:
pd.read_csv("examples/ex4.csv", skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


- Xử lý giá trị bị thiếu là một phần quan trọng và thường khá tinh tế trong quá trình đọc tệp. Dữ liệu bị thiếu thường hoặc là không xuất hiện (chuỗi rỗng), hoặc được đánh dấu bằng một giá trị đại diện (sentinel / placeholder). Mặc định, pandas sử dụng một tập hợp các giá trị đại diện thường gặp, chẳng hạn như NA và NULL:

In [19]:
!cat examples/ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo

In [20]:
result = pd.read_csv("examples/ex5.csv")

result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


- Hãy nhớ rằng pandas hiển thị các giá trị bị thiếu dưới dạng NaN, vì vậy trong kết quả chúng ta có hai giá trị null hoặc bị thiếu:

In [21]:
pd.isna(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


- Tùy chọn `na_values` chấp nhận một dãy (sequence) các chuỗi để bổ sung vào danh sách mặc định những chuỗi được nhận diện là giá trị bị thiếu:

In [22]:
result = pd.read_csv("examples/ex5.csv", na_values=["NULL"])

result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


- pandas.read_csv có một danh sách gồm nhiều cách biểu diễn giá trị NA mặc định, nhưng các giá trị mặc định này có thể bị vô hiệu hóa bằng tùy chọn `keep_default_na`:

In [23]:
result2 = pd.read_csv("examples/ex5.csv", keep_default_na=False)

result2

,something,a,b,c,d,message
0,one,1,2,3,4,NA
1,two,5,6,,8,world
2,three,9,10,11,12,foo


In [24]:

result2.isna()

,something,a,b,c,d,message
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False


In [25]:
result3 = pd.read_csv("examples/ex5.csv", keep_default_na=False, na_values=["NA"])
result3

,something,a,b,c,d,message
0,one,1,2,3,4,NaN
1,two,5,6,,8,world
2,three,9,10,11,12,foo


In [26]:
result3.isna()

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,False,False,False
2,False,False,False,False,False,False


> Các giá trị đại diện cho NA khác nhau có thể được chỉ định cho từng cột trong một dictionary:

In [27]:
sentinels = {"message": ["foo", "NA"], "something": ["two"]}
pd.read_csv("examples/ex5.csv", na_values=sentinels, keep_default_na=False)

,something,a,b,c,d,message
0,one,1,2,3,4,NaN
1,NaN,5,6,,8,world
2,three,9,10,11,12,NaN


> Table 6-2 lists some frequently used options in pandas.read_csv.

| Argument        | Description |
|-----------------|-------------|
| path            | String indicating filesystem location, URL, or file-like object. |
| sep or delimiter| Character sequence or regular expression to use to split fields in each row. |
| header          | Row number to use as column names; defaults to 0 (first row), but should be None if there is no header row. |
| index_col       | Column numbers or names to use as the row index in the result; can be a single name/number or a list of them for a hierarchical index. |
| names           | List of column names for result. |
| skiprows        | Number of rows at beginning of file to ignore or list of row numbers (starting from 0) to skip. |
| na_values       | Sequence of values to replace with NA. They are added to the default list unless `keep_default_na=False` is passed. |
| keep_default_na | Whether to use the default NA value list or not (True by default). |
| comment         | Character(s) to split comments off the end of lines. |
| parse_dates     | Attempt to parse data to datetime; False by default. If True, will attempt to parse all columns. Otherwise, can specify a list of column numbers or names to parse. If element of list is tuple or list, will combine multiple columns together and parse to date. |
| keep_date_col   | If joining columns to parse date, keep the joined columns; False by default. |
| converters      | Dictionary containing column number or name mapping to functions (e.g., {"foo": f} would apply the function f to all values in the "foo" column). |
| dayfirst        | When parsing potentially ambiguous dates, treat as international format (e.g., 7/6/2012 → June 7, 2012); False by default. |
| date_parser     | Function to use to parse dates. |
| nrows           | Number of rows to read from beginning of file (not counting the header). |
| iterator        | Return a TextFileReader object for reading the file piecemeal. This object can also be used with the `with` statement. |
| chunksize       | For iteration, size of file chunks. |
| skip_footer     | Number of lines to ignore at end of file. |
| verbose         | Print various parsing information, like the time spent in each stage of the file conversion and memory use information. |
| encoding        | Text encoding (e.g., "utf-8" for UTF-8 encoded text). Defaults to "utf-8" if None. |
| squeeze         | If the parsed data contains only one column, return a Series. |
| thousands       | Separator for thousands (e.g., "," or "."); default is None. |
| decimal         | Decimal separator in numbers (e.g., "." or ","); default is ".". |
| engine          | CSV parsing and conversion engine to use; can be one of "c", "python", or "pyarrow". The default is "c", though the newer "pyarrow" engine can parse some files much faster. The "python" engine is slower but supports some features that the other engines do not. |


## Reading Text Files in Pieces

- Khi xử lý các tệp rất lớn hoặc khi cần tìm ra tập hợp tham số phù hợp để xử lý đúng một tệp lớn, bạn có thể chỉ muốn đọc một phần nhỏ của tệp hoặc lặp qua những khối nhỏ hơn của tệp.

> Trước khi chúng ta xem xét một tệp lớn, hãy điều chỉnh thiết lập hiển thị của pandas để gọn gàng hơn:

In [28]:
pd.options.display.max_rows = 10

> Giờ bạn có thể:

In [29]:
result = pd.read_csv("examples/ex6.csv")

result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


- Dấu ba chấm `...` cho biết rằng các hàng ở giữa của **DataFrame** đã được lược bỏ.

- Nếu bạn chỉ muốn đọc một số lượng nhỏ các hàng (tránh việc đọc toàn bộ tệp), hãy chỉ định bằng tham số `nrows`:

In [30]:
pd.read_csv("examples/ex6.csv", nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


- Để đọc một tệp theo từng phần, hãy chỉ định `chunksize` dưới dạng số lượng hàng:

In [31]:
chunker = pd.read_csv("examples/ex6.csv", chunksize=1000)
type(chunker)

pandas.io.parsers.readers.TextFileReader

- Đối tượng TextFileReader được trả về bởi pandas.read_csv cho phép bạn lặp qua các phần của tệp theo `chunksize`. 
- Ví dụ, chúng ta có thể lặp qua tệp ex6.csv, đồng thời tổng hợp số lần xuất hiện của các giá trị trong cột "key", như sau:

In [32]:
chunker = pd.read_csv("examples/ex6.csv", chunksize=1000)
tot = pd.Series([], dtype='int64')
for piece in chunker:
    tot = tot.add(piece["key"].value_counts(), fill_value=0)
tot = tot.sort_values(ascending=False)

> Khi đó chúng ta sẽ có:

In [33]:
tot[:10]

key
E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

- TextFileReader cũng được trang bị phương thức `get_chunk`, cho phép bạn đọc từng phần với kích thước tùy ý.

## Writing Data to Text Format

> Dữ liệu cũng có thể được xuất ra định dạng có ký tự phân tách. Hãy xem xét một trong những tệp CSV đã được đọc trước đó:

In [34]:
data = pd.read_csv("examples/ex5.csv")
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


- Sử dụng phương thức `to_csv` của DataFrame, chúng ta có thể ghi dữ liệu ra một tệp được phân tách bằng dấu phẩy (CSV):

In [35]:
data.to_csv("examples/out.csv")
!cat examples/out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


- Tất nhiên, bạn cũng có thể sử dụng các ký tự phân tách khác (ghi ra **sys.stdout** để in kết quả dạng văn bản ra **console** thay vì ghi vào tệp):

In [36]:
import sys
data.to_csv(sys.stdout, sep="|")

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


- Các giá trị bị thiếu sẽ xuất hiện dưới dạng chuỗi rỗng trong đầu ra. Bạn có thể muốn biểu thị chúng bằng một giá trị đại diện khác:

In [37]:
data.to_csv(sys.stdout, na_rep="NULL")


,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


- Khi không chỉ định thêm tùy chọn nào khác, cả nhãn hàng (row labels) và nhãn cột (column labels) đều sẽ được ghi ra. Cả hai đều có thể bị vô hiệu hóa:

In [38]:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


> Bạn cũng có thể ghi chỉ một tập con các cột, và theo thứ tự mà bạn chọn:

In [39]:
data.to_csv(sys.stdout, index=False, columns=["a", "b", "c"])

a,b,c
1,2,3.0
5,6,
9,10,11.0


## Working with Other Delimited Formats

- Có thể nạp hầu hết các dạng dữ liệu dạng bảng từ ổ đĩa bằng các hàm như `pandas.read_csv`. Tuy nhiên, trong một số trường hợp, có thể cần phải xử lý thủ công. 
- Việc nhận được một tệp có một hoặc nhiều dòng bị lỗi định dạng khiến `pandas.read_csv` gặp sự cố là điều không hiếm.
- Để minh họa các công cụ cơ bản, hãy xem xét một tệp CSV nhỏ:

In [40]:
!cat examples/ex7.csv

"a","b","c"
"1","2","3"
"1","2","3"


- Đối với bất kỳ tệp nào có ký tự phân tách chỉ gồm một ký tự, bạn có thể sử dụng mô-đun csv tích hợp sẵn của Python. Để dùng nó, hãy truyền vào một tệp đã mở hoặc một đối tượng giống tệp (file-like object) cho csv.reader:

In [41]:
import csv
f = open("examples/ex7.csv")
reader = csv.reader(f)

- Lặp qua reader giống như lặp qua một tệp sẽ trả về các danh sách giá trị, với mọi ký tự ngoặc kép được loại bỏ:

In [42]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


In [43]:
f.close()

- Từ đó, phần việc còn lại tùy thuộc vào bạn để thực hiện các thao tác wrangling cần thiết nhằm đưa dữ liệu về dạng bạn cần. Hãy làm từng bước một. Trước hết, ta đọc tệp vào một danh sách các dòng:

In [44]:
with open("examples/ex7.csv") as f: 
    lines = list(csv.reader(f))

> Sau đó, chúng ta tách các dòng thành dòng tiêu đề và các dòng dữ liệu:

In [45]:
header, values = lines[0], lines[1:]

- Sau đó, chúng ta có thể tạo một từ điển (dictionary) các cột dữ liệu bằng cách sử dụng dictionary comprehension và biểu thức zip(*values) (hãy cẩn thận vì thao tác này sẽ tốn rất nhiều bộ nhớ với các tệp lớn), thao tác này sẽ hoán chuyển các hàng thành các cột:

In [46]:
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

- Các tệp CSV có nhiều dạng khác nhau. Để định nghĩa một định dạng mới với dấu phân cách khác, quy ước đặt chuỗi trong dấu ngoặc kép khác, hoặc ký tự kết thúc dòng khác, chúng ta có thể định nghĩa một lớp con đơn giản của `csv.Dialect`:

```python


In [47]:
obj = """
{"name": "Wes",
 "cities_lived": ["Akron", "Nashville", "New York", "San Francisco"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 34, "hobbies": ["guitars", "soccer"]},
              {"name": "Katie", "age": 42, "hobbies": ["diving", "art"]}]
}
"""

In [48]:
import json
result = json.loads(obj)
result

{'name': 'Wes',
 'cities_lived': ['Akron', 'Nashville', 'New York', 'San Francisco'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 34, 'hobbies': ['guitars', 'soccer']},
  {'name': 'Katie', 'age': 42, 'hobbies': ['diving', 'art']}]}

In [49]:
asjson = json.dumps(result)
asjson

'{"name": "Wes", "cities_lived": ["Akron", "Nashville", "New York", "San Francisco"], "pet": null, "siblings": [{"name": "Scott", "age": 34, "hobbies": ["guitars", "soccer"]}, {"name": "Katie", "age": 42, "hobbies": ["diving", "art"]}]}'

In [50]:
siblings = pd.DataFrame(result["siblings"], columns=["name", "age"])
siblings

,name,age
0,Scott,34
1,Katie,42


In [52]:
!cat examples/example.json

[{"a": 1, "b": 2, "c": 3},
 {"a": 4, "b": 5, "c": 6},
 {"a": 7, "b": 8, "c": 9}]


In [53]:
data = pd.read_json("examples/example.json")
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


In [54]:
data.to_json(sys.stdout)
data.to_json(sys.stdout, orient="records")

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]

In [58]:
pip install lxml


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 47.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [59]:
tables = pd.read_html("examples/fdic_failed_bank_list.html")
len(tables)
failures = tables[0]
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


In [60]:
close_timestamps = pd.to_datetime(failures["Closing Date"])
close_timestamps.dt.year.value_counts()

Closing Date
2010    157
2009    140
2011     92
2012     51
2008     25
       ... 
2004      4
2001      4
2007      3
2003      3
2000      2
Name: count, Length: 15, dtype: int64

In [61]:
from lxml import objectify

path = "datasets/mta_perf/Performance_MNR.xml"
with open(path) as f:
    parsed = objectify.parse(f)
root = parsed.getroot()

In [62]:
data = []

skip_fields = ["PARENT_SEQ", "INDICATOR_SEQ",
               "DESIRED_CHANGE", "DECIMAL_PLACES"]

for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

In [63]:
perf = pd.DataFrame(data)
perf.head()

,AGENCY_NAME,INDICATOR_NAME,DESCRIPTION,PERIOD_YEAR,PERIOD_MONTH,CATEGORY,FREQUENCY,INDICATOR_UNIT,YTD_TARGET,YTD_ACTUAL,MONTHLY_TARGET,MONTHLY_ACTUAL
0,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 m...,2008,1,Service Indicators,M,%,95.0,96.9,95.0,96.9
1,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 m...,2008,2,Service Indicators,M,%,95.0,96.0,95.0,95.0
2,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 m...,2008,3,Service Indicators,M,%,95.0,96.3,95.0,96.9
3,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 m...,2008,4,Service Indicators,M,%,95.0,96.8,95.0,98.3
4,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 m...,2008,5,Service Indicators,M,%,95.0,96.6,95.0,95.8


In [64]:
perf2 = pd.read_xml(path)
perf2.head()

,INDICATOR_SEQ,PARENT_SEQ,AGENCY_NAME,INDICATOR_NAME,DESCRIPTION,PERIOD_YEAR,PERIOD_MONTH,CATEGORY,FREQUENCY,DESIRED_CHANGE,INDICATOR_UNIT,DECIMAL_PLACES,YTD_TARGET,YTD_ACTUAL,MONTHLY_TARGET,MONTHLY_ACTUAL
0,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 m...,2008,1,Service Indicators,M,U,%,1,95.00,96.90,95.00,96.90
1,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 m...,2008,2,Service Indicators,M,U,%,1,95.00,96.00,95.00,95.00
2,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 m...,2008,3,Service Indicators,M,U,%,1,95.00,96.30,95.00,96.90
3,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 m...,2008,4,Service Indicators,M,U,%,1,95.00,96.80,95.00,98.30
4,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 m...,2008,5,Service Indicators,M,U,%,1,95.00,96.60,95.00,95.80


In [65]:
frame = pd.read_csv("examples/ex1.csv")
frame
frame.to_pickle("examples/frame_pickle")

In [66]:
pd.read_pickle("examples/frame_pickle")

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [67]:
!rm examples/frame_pickle

In [ ]:
fec = pd.read_parquet('datasets/fec/fec.parquet')

In [ ]:
xlsx = pd.ExcelFile("examples/ex1.xlsx")

In [ ]:
xlsx.sheet_names

In [ ]:
xlsx.parse(sheet_name="Sheet1")

In [ ]:
xlsx.parse(sheet_name="Sheet1", index_col=0)

In [ ]:
frame = pd.read_excel("examples/ex1.xlsx", sheet_name="Sheet1")
frame

In [ ]:
writer = pd.ExcelWriter("examples/ex2.xlsx")
frame.to_excel(writer, "Sheet1")
writer.close()

In [ ]:
frame.to_excel("examples/ex2.xlsx")

In [ ]:
!rm examples/ex2.xlsx

In [ ]:
!rm -f examples/mydata.h5

In [ ]:
frame = pd.DataFrame({"a": np.random.standard_normal(100)})
store = pd.HDFStore("examples/mydata.h5")
store["obj1"] = frame
store["obj1_col"] = frame["a"]
store

In [ ]:
store["obj1"]

In [ ]:
store.put("obj2", frame, format="table")
store.select("obj2", where=["index >= 10 and index <= 15"])
store.close()

In [ ]:
frame.to_hdf("examples/mydata.h5", "obj3", format="table")
pd.read_hdf("examples/mydata.h5", "obj3", where=["index < 5"])

In [ ]:
import os
os.remove("examples/mydata.h5")

In [ ]:
import requests
url = "https://api.github.com/repos/pandas-dev/pandas/issues"
resp = requests.get(url)
resp.raise_for_status()
resp

In [ ]:
data = resp.json()
data[0]["title"]

In [ ]:
issues = pd.DataFrame(data, columns=["number", "title",
                                     "labels", "state"])
issues

In [ ]:
import sqlite3

query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d INTEGER
);"""

con = sqlite3.connect("mydata.sqlite")
con.execute(query)
con.commit()

In [ ]:
data = [("Atlanta", "Georgia", 1.25, 6),
        ("Tallahassee", "Florida", 2.6, 3),
        ("Sacramento", "California", 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"

con.executemany(stmt, data)
con.commit()

In [ ]:
cursor = con.execute("SELECT * FROM test")
rows = cursor.fetchall()
rows

In [ ]:

cursor.description
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

In [ ]:
import sqlalchemy as sqla
db = sqla.create_engine("sqlite:///mydata.sqlite")
pd.read_sql("SELECT * FROM test", db)

In [ ]:
!rm mydata.sqlite